In [ ]:
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
import scanpy as sc
import pandas as pd
import torch
import scipy
import time
from model import *
from utils import *

In [ ]:
scdata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_sc_counts.csv', index_col = 0)
scdata.head(5)

In [ ]:
scdata.shape

In [ ]:
stdata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_counts.csv', index_col = 0)
stdata.head(5)

In [ ]:
stdata.shape

In [ ]:
scdata = scdata.T
stdata = stdata.T

In [ ]:
stgtcelltype = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_celltype.csv', index_col = 0)
stgtcelltype.head(5)

In [ ]:
stgtcelltype.shape

In [ ]:
scdata.head(5)

In [ ]:
stdata.head(5)

In [ ]:
stdata.shape

In [ ]:
spcoor = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/simu_st_metadata.csv', index_col=0)
spcoor.head(5)

In [ ]:
spcoor.shape

In [ ]:
scmetadata = pd.read_csv('/home/ehtesamul/sc_st/data/mousedata_2020/E1z2/metadata.csv', index_col = 0)
scmetadata.head(5)

In [ ]:
scmetadata.shape

In [ ]:
adata = sc.AnnData(scdata, obs=scmetadata)
adata

In [ ]:
adata.X

In [ ]:
adata.var_names

In [ ]:
adata.obs_names

In [ ]:
scdata.head(5)

In [ ]:
scmetadata

In [ ]:
new_scdata = pd.DataFrame(adata.X,index=adata.obs_names,columns=adata.var_names)
new_scdata.head(5)

In [ ]:
adata.obs['embryo']

In [ ]:
scdata.head(5)

In [ ]:
new_scdata.head(5)

In [ ]:
adata.obs

In [ ]:
scdata = new_scdata
stadata = sc.AnnData(stdata)
sc.pp.normalize_total(stadata)
sc.pp.log1p(stadata)
stdata = pd.DataFrame(stadata.X, index = stadata.obs_names, columns = stadata.var_names)

In [ ]:
adata.obsm['spatial'] = scmetadata[['x_global', 'y_global']].values

In [ ]:
adata.obs

In [ ]:
spcoor

In [ ]:
adata.obsm['spatial']

In [ ]:
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
adata.obs

In [ ]:
sc.pp.calculate_qc_metrics(stadata,percent_top=None, log1p=False, inplace=True)


In [ ]:
dp = 1- adata.obs['n_genes_by_counts'].median()/stadata.obs['n_genes_by_counts'].median()
dp

In [ ]:
class setting( object ):
    pass
seed_all(2022)
opt= setting()
setattr(opt, 'device', 'cuda:0') # device
setattr(opt, 'outf', 'log/test') # folder to save log files
setattr(opt, 'n_genes', 351) # number of genes for the input
setattr(opt, 'no_bn', False) # duplicated
setattr(opt, 'lr', 0.002) # learning rate
setattr(opt, 'sigma', 3)  # the spatial variance parameter in the Gaussian function
setattr(opt, 'alpha', 0.8) # MMD loss weight default:0.8
setattr(opt, 'verbose', True) # verbose
setattr(opt, 'mmdbatch', 1000) # batch for MMD loss
setattr(opt, 'dp', dp) # dropout rate for ST data

In [ ]:
opt.device

In [ ]:
if not os.path.exists(opt.outf):
    os.makedirs(opt.outf)

In [ ]:
spcoor

In [ ]:
stgtcelltype

In [ ]:
stadata.obs

In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc

# Set figure size
plt.figure(figsize=(10, 5))

# Create the violin plots
sc.pl.violin(stadata, ['n_genes_by_counts', 'total_counts'], 
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'],jitter=0.4, multi_panel=True)

In [ ]:
st_neighbor = scipy.spatial.distance.cdist(spcoor, spcoor)
sigma = 3
st_neighbor = np.exp(-st_neighbor ** 2/ (2*sigma**2))/ (np.sqrt(2*np.pi)*sigma)

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(spcoor.values[:, 0], spcoor.values[:, 1], c=np.log1p(st_neighbor[400, :]), s=20)

In [ ]:
class setting( object ):
    pass
seed_all(2022)
opt= setting()
setattr(opt, 'device', 'cuda:0')
setattr(opt, 'outf', 'log/Mouse_E1z2')
setattr(opt, 'n_genes', 351)
setattr(opt, 'no_bn', False)
setattr(opt, 'lr', 0.002)
setattr(opt, 'sigma', 3)
setattr(opt, 'alpha', 0.8)
setattr(opt, 'verbose', True)
setattr(opt, 'mmdbatch', 1000)
setattr(opt, 'dp', dp)

In [ ]:
%%time
testmodel = SOmodel(opt)
testmodel.togpu()
loss_curve = testmodel.train_wholedata(400, torch.tensor(scdata.values).float(), torch.tensor(stdata.values).float(), torch.tensor(spcoor.values).float())

In [ ]:
plt.figure(figsize=(10,6))
skip = 10
N = len(loss_curve['E']) - skip
plt.plot(np.arange(N), loss_curve['E'][skip:], label='E')
plt.plot(np.arange(N), loss_curve['E_pred'][skip:], label='E_pred')
plt.plot(np.arange(N), loss_curve['E_circle'][skip:], label='E_circle')
plt.plot(np.arange(N), loss_curve['E_mmd'][skip:], label='E_mmd')
plt.legend()

In [ ]:
testmodel.modeleval()
scembedding = testmodel.netE(torch.tensor(scdata.values, dtype=torch.float32).cuda())
stembedding = testmodel.netE(torch.tensor(stdata.values, dtype=torch.float32).cuda())
netst2sc = F.softmax(stembedding.mm(scembedding.t()),dim=1).detach().cpu().numpy()
netsc2st = F.softmax(scembedding.mm(stembedding.t()), dim=1).detach().cpu().numpy()

In [ ]:
netst2sc.shape

In [ ]:
netsc2st.shape

In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
dataset_name = 'E1z2'
result_dir = f'result/{timestamp}_{dataset_name}'
os.makedirs(result_dir, exist_ok=True)

np.save(f'{result_dir}/st2sc_mapping.npy', netst2sc)
np.save(f'{result_dir}/sc2st_mapping.npy', netsc2st)
np.save(f'{result_dir}/sc_embedding.npy', scembedding.detach().cpu().numpy())
np.save(f'{result_dir}/st_embedding.npy', stembedding.detach().cpu().numpy())

In [ ]:
testmodel.save()

In [ ]:
def all_coord(mappingmtx,spcoor):
    mappingnorm = mappingmtx.div(mappingmtx.sum(axis=1), axis=0)

    spaotsc_coord = np.zeros([mappingnorm.shape[0],2])
    spaotsc_coord[:,0] = np.matmul(mappingnorm.values,spcoor.coord_x.values)
    spaotsc_coord[:,1] = np.matmul(mappingnorm.values,spcoor.coord_y.values)
    return spaotsc_coord

def k_coord(mappingmtx,spcoor, k=1):
    maskmap = np.zeros_like(mappingmtx)

    argmask = np.argsort(1/mappingmtx.values,axis=1)

    for i in range(argmask.shape[0]):
        for j in range(k):
            maskmap[i,argmask[i,j]]=1

    mappingnorm = (maskmap*mappingmtx).div((maskmap*mappingmtx).sum(axis=1), axis=0)

    spaotsc_coord = np.zeros([mappingnorm.shape[0],2])
    spaotsc_coord[:,0] = np.matmul(mappingnorm.values,spcoor.coord_x.values)
    spaotsc_coord[:,1] = np.matmul(mappingnorm.values,spcoor.coord_y.values)
    return spaotsc_coord

In [ ]:
adata.obsm['SpatialDA'] = all_coord(pd.DataFrame(netsc2st, index=adata.obs_names, columns=stadata.obs_names), spcoor)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

In [190]:
def gen_sc_adj_matrix(sc_embeddings):
    #change to numpy array if tensor
    if isinstance(sc_embeddings, torch.Tensor):
        embeddings = sc_embeddings.detach().cpu().numpy()
    else:
        embeddings = sc_embeddings

    #calculate inner product
    adj_matrix = np.matmul(embeddings, embeddings.T)

    #normalize it
    # row_sums = adj_matrix.sum(axis=1)
    # normalized_adj_matrix = adj_matrix / row_sums[:, np.newaxis]
    normalized_adj_matrix = adj_matrix / adj_matrix.sum(axis=1, keepdims=True)

    return normalized_adj_matrix

def recover_coord_from_adj_matrix(adj_matrix, n_components=2):
    from sklearn.manifold import MDS

    distance_matrix = 1 - adj_matrix
    distance_matrix = 0.5 * (distance_matrix + distance_matrix.T)

    #apply mds to recover coordinates
    mds = MDS(n_components=n_components,
              dissimilarity='precomputed',
              random_state=42,
              normalized_stress='auto')
    
    coordinates = mds.fit_transform(distance_matrix)

    return coordinates

In [191]:
#generate sc embeddings
scembedding = testmodel.netE(torch.tensor(scdata.values, dtype = torch.float32).cuda())

#generate sc adjacency matrix from embeddings
sc_adjacency_matrix = gen_sc_adj_matrix(scembedding)

#recover coordinates from adjacency matrix
recovered_coordinates = recover_coord_from_adj_matrix(sc_adjacency_matrix)

#add to AnnData object
adata.obsm['MDSSpatial'] = recovered_coordinates

In [193]:
adata.obsm['MDSSpatial']

array([[-0.16331454,  0.75031373],
       [ 0.43977416,  0.16300028],
       [-0.50681446, -0.49988661],
       ...,
       [-0.23938644, -0.60675617],
       [ 0.40582559,  0.06321056],
       [ 0.44911275, -0.60269493]])

In [194]:
adata.obsm['SpatialDA']

array([[1627.29216741, 1349.74699412],
       [1625.44197922, 1348.43905111],
       [1625.99117369, 1336.13772766],
       ...,
       [1550.36125401, 1536.50573694],
       [1556.35831   , 1521.93157017],
       [1553.71221347, 1518.64542879]])

In [196]:
def distance_from_adjacency(adj_matrix, sigma = 3.0, epsilon=1e-10):
    #assuming adjacency matrix is normalized so that each row sums to 1

    #adjacecny values should be pisitive to avoid log(0)
    adj_cleaned = np.maximum(adj_matrix, epsilon)

    #apply inverse gaussian kernel transformation
    #D_{ij} = sqrt(-2*sigma^2 * ln(S^{SC}_{ij} + epsilon))
    distance_squared = -2 * sigma**2 * np.log(adj_cleaned)

    #ensure distances are non-negative
    distance_squared = np.maximum(distance_squared, 0)

    #take square root to get the distances
    distance_matrix = np.maximum(distance_squared, 0)

    #take square root to get distance
    distance_matrix = np.sqrt(distance_squared)

    #make the matrix symmetric (average with its transpose)
    distance_matrix = 0.5 * (distance_matrix + distance_matrix.T)

    return distance_matrix


def coordinates_from_adjacency(adj_matrix, sigma=3.0, n_components=2):
    from sklearn.manifold import MDS
    #recover distance from adjacency
    distance_matrix = distance_from_adjacency(adj_matrix,  sigma)

    #appl mds to recover coordinates
    mds = MDS(n_components=n_components,
              dissimilarity='precomputed',
              random_state = 42,
              normalized_stress= 'auto')
    
    coordinates = mds.fit_transform(distance_matrix)

    return coordinates

def sc_coordinates_from_embeddings(sc_embeddings, sigma=3.0):
    if isinstance(sc_embeddings, torch.Tensor):
        embeddings = sc_embeddings.detach().cpu().numpy()
    else:
        embeddings = sc_embeddings

    #calculate SC spatial adjacency matrix using inner products 
    sc_adjacency_matrix = np.matmul(embeddings, embeddings.T)

    #normalize rows to sum to 1
    row_sums = sc_adjacency_matrix.sum(axis=1, keepdims=True)
    sc_adjacency_matrix_norm = sc_adjacency_matrix / row_sums

    #recover coordinates using inverse gaussian kernel and MDS
    coordinates = coordinates_from_adjacency(sc_adjacency_matrix_norm, sigma = 3.0)

    return coordinates, sc_adjacency_matrix_norm

In [ ]:
#generate SC embeddings
testmodel.modeleval()
scembedding = testmodel.netE(torch.tensor(scdata.values, dtype=torch.float32).cuda())

#get coordinates from embeddings 
recovered_coords, sc_adjacency_matrix = sc_coordinates_from_embeddings(scembedding, sigma = opt.sigma)

In [184]:
adj_matrix = np.array([
    [2, 4, 6], 
    [1, 3, 5], 
    [7, 8, 9]
])


In [185]:
row_sums = adj_matrix.sum(axis=1)
row_sums


array([12,  9, 24])

In [186]:
adj_matrix = adj_matrix / row_sums[:, np.newaxis]

In [188]:
row_sums[np.newaxis, :]

array([[12,  9, 24]])

In [195]:
spcoor

,coord_x,coord_y
ST_12,1581.242732,1329.313864
ST_13,1586.662026,1329.313864
ST_42,1581.242732,1334.863919
ST_43,1586.662026,1334.863919
ST_49,1619.177788,1334.863919
...,...,...
ST_1182,1581.242732,1545.766004
ST_1183,1586.662026,1545.766004
ST_1184,1592.081319,1545.766004
ST_1185,1597.500613,1545.766004
